## Suman Thapaliya


# Visualization

### Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import osmnx as ox
import geopandas as gpd

### Load the Prepared CSV File

In [ ]:
df= pd.read_csv('final.csv')
df.head(5)

#  Map
In our first visualization, we did map plot using carto-positron to show how many people were victims of railway accident across Europe from 2004 to 2015. Number of Victims or number of People affected in our visualization means total number of people who were either injured or died due to railway accident.

In [ ]:
df_aggregated_4= df.groupby(['Country']).agg({'Value': 'sum'}).reset_index()

In [ ]:
european_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                      'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
                      'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
                      'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
                      'Slovenia', 'Spain', 'Sweden', 'United Kingdom']

boundaries = []
for country in european_countries:
    boundary = ox.geocode_to_gdf(country)
    boundaries.append(boundary)

gdf_europe = gpd.GeoDataFrame(pd.concat(boundaries, ignore_index=True))

gdf_europe = gdf_europe.merge(df_aggregated_4, left_on='name', right_on='Country', how='left')

fig = px.choropleth_mapbox(gdf_europe,
                           geojson=gdf_europe.geometry,
                           locations=gdf_europe.index,
                           color='Value',
                           hover_name='Country',  
                           hover_data={'Country': True, 'Value': True},  
                           mapbox_style="carto-positron",
                           center={"lat": 51.1634, "lon": 10.4477},
                           zoom=3,
                           color_continuous_scale='Cividis',
                           labels={'Value': 'Number of Victims'})

fig.update_traces(hoverinfo='text',
                  hovertemplate='<b>%{hovertext}</b><br>'
                                'Number of Victims: %{customdata[1]:,.0f}<extra></extra>',
                  customdata=gdf_europe[['Country', 'Value']].values)


fig.update_traces(hoverlabel=dict(bgcolor="lightblue",  
                                font_size=12,  
                                font_family="Arial")) 



fig.update_layout(title='Number of Victims in Railway Accidents Across Europe (2004-2015)',
                  title_x=0.5,
                  margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.show()

### Interpretation
At first glance, we can observe that Poland has highest number of people affected by railyway accident compare to other EU countries. Germany, Hungary, and Romania has also affected alot where as Ireland and Luxembourg is safest while it comes to railway safety. It is interesting to see that two countries who possess higher risk factor shares their borders i.e. Germany and Poland.

# Time Series
For this Visualization, we used time series plot to understand the pattern how the individual country has worked to make railway system more safe over the years.

In [ ]:
df_aggregated = df.groupby(['Year', 'Country']).agg({'Value': 'sum'}).reset_index()

In [ ]:
fig = px.line(
    df_aggregated,
    x='Year',
    y='Value',
    color='Country',  
    labels={
        'Year': 'Year',
        'Value': 'People Affected',
        'Country': 'Country'
    },
)

fig.update_xaxes(
    dtick="M12",  
    tickformat="%Y",  
    #ticklabelmode="period"  
)


fig.update_yaxes(
    title_text='People Affected',
    title_font=dict(color='green')  
)

fig.update_xaxes(
    title_text='Year',
    title_font=dict(color='green')  
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=5, label='5Y', step='year', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),  
        type='date' 
    ),
    title={
        'text': 'People Affected in Railway Accident',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 23}
    },
    
    annotations=[
        go.layout.Annotation(
            text="European Country from 2004 to 2015",
            xref="paper", yref="paper",
            x=0.56, y=1.20,
            showarrow=False,
            font=dict(size=14, color="grey"),
            align="center"
        )
    ],
    width=1150,  
    height=500,  
    margin=dict(t=120)  
)

fig.show()


### Interpretation
From the map visualization, we can say that poland railway system has serious issue when it comes to safety. But from the time series plot, we observe that Poland had been working on to make railway a safer transportation mode over the years. Compare to 2004, the number of people affected in 2015 has been half which is great improvement. Nevertheless, we can see that Poland is still at high risk in railway transportation in compare to other EU countries. Another intresting observation is Spain. In 2013, there was major railway accident which affected 191 peoples. Belgium has also similar incident in 2010.

# Stacked Bar Plot
In stacked Bar plot, we want to know which type of railway accident has affected the people the most.

In [ ]:
df_aggregated_1 = df.groupby(['Year', 'Accident']).agg({'Value': 'sum'}).reset_index()

In [ ]:
color_scale = px.colors.sequential.Viridis 

fig = px.bar(
    df_aggregated_1,
    x='Year',
    y='Value',
    color='Accident',
    title='Stacked Bar Plot of Accident Types by Year',
    labels={'Value': 'People Affected', 'Year': 'Year', 'Accident': 'Accident Type'},
    barmode='stack',
    color_continuous_scale=color_scale
)

fig.update_xaxes(
    title_text='Year',
    title_font=dict(color='green')  
)

fig.update_yaxes(
    title_text='People Affected',
    title_font=dict(color='green')  
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=5, label='5Y', step='year', stepmode='backward'),
                dict(step='all')
            ])
        ),
        type='date'  
    ),
    title={
        'text': 'People Affected by Accident Types',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 25}
    },
    annotations=[
        dict(
            x=0.53,
            y=1.2,
            xref='paper',
            yref='paper',
            text='From 2004 to 2015 of European Countries',
            showarrow=False,
            font=dict(size=14, color='grey'),
            align='center'
        )
    ],
    width=1150,  
    height=500,  
    margin=dict(t=130)  
)

fig.show()

### Interpretation
LVLCROS i.e. Level Crossing accident and RSTK_MOT i.e. accidents to persons caused by rolling stock in motion accident were two major type of railway accident which affected the people most. People is our case refers to employee of the railway department, and passanger mainly. These two types of accident are dangerous as it interact with people directly. It is good to notice that there has been few accident due to derailment as it almost depends upon technology and engineering. Nevertheless, there has been an major derailment accident in 2013 which is because of Spain railway accident that we observed in time series plot. To prove it, I have shown in Further investigation.

### Further Investigation
In 2013, Spain has major detrailment accident which affected 149 people.

In [ ]:
df_aggregated= df.groupby(['Accident','Country','Year']).agg({'Value': 'sum'}).reset_index()
filtered_df = df_aggregated[df_aggregated['Country'] == 'Spain']
filtered_df = filtered_df[filtered_df['Accident'] == 'DERAIL']
filtered_df

# Bar Plot
We wanted to deep dive into subset of our dataset to observe specific information. we choosed to visualized the total number of people who lost their life in between 2004 to 2015 based on country. we know that Poland possess highest risk when it comes to railway safety. If is it so, then Poland should be the country where people are not safe to use railway transport in compare to other EU countries.  

In [ ]:
df_killed_victims = df[df['Victim'] == 'KIL']

In [ ]:
df_aggregated_3 = df_killed_victims.groupby(['Country', 'Victim']).agg({'Value': 'sum'}).reset_index()

In [ ]:
df_aggregated_3 = df_aggregated_3.sort_values(by='Value', ascending=True)

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Value', y='Country', data=df_aggregated_3, palette='viridis')

plt.title('Number of People Die in Railway Accident(2004-2015)', color='blue')
plt.xlabel('Number of Victims Killed', color='blue')
plt.ylabel('', color='blue')

for p in ax.patches:
    ax.text(p.get_width() + 0.02 * p.get_width(), p.get_y() + p.get_height() / 2, '{:d}'.format(int(p.get_width())), ha='left', va='center', color='black')

sns.despine()

ax.xaxis.set_visible(False)

fig = go.Figure()

### Interpretation
From Bar plot, it is obvious that Poland is the country who lost most people due to railway accident. It is worth while noticing that countries like Estonia, and Lithuania posses less risk comparing to other developed EU countries when it comes to railway transportation. In overall, many EU countries like Netherlands, Denmark, Ireland, and Finland posses minimum risk to use public transportation like railway.

# Summary
Interpretation of all above visualizations gives imporatant insights about the safety factor in railway industry of European countries. we can come to conclusion that Poland holds the high risk in railway transportation for public safety. we also observe that they are improving over the past decades and created a low risk environment in railway system. Moreover, we saw that Germany posses a bit of risk in comapre to other EU developed countries and Luxembourg has the safest railway system in Europe in between 2004 to 2015. It is worth to mention that UK was also part of EU during 2004 to 2015. so, we included it and it posses moderate risk compare to others.

# Limitation
We had many great insights of railway accident of EU in between 2004 to 2015. Nevertheless, if we have to order which country possess highest risk to use railway system, then we would hesitate to rank it. For instance, Germany is second highest in terms of risk factor, but we don't know the other factor like how many people were affcted per million or thousands. As we know that Romania total population is 1/4 th of Germany. If so, Germany possess half risk than Romania possess, but the figure says the other story.